In [1]:
import requests
import unittest
import logging
import pandas as pd

In [2]:
URL = 'http://127.0.0.1:8000/'
HEADERS = {'Content-type': 'application/json', 'Accept': 'text/plain'}


In [3]:
# Настройка логгера
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)


In [4]:
catalog_names = pd.read_parquet('../catalog_names.parquet')
items = pd.read_parquet('../items.parquet')
result = pd.merge(items,
                   catalog_names[catalog_names["type"] == "track"], 
                   left_on="track_id", right_on="id", how="inner").groupby(['id', 'name']).agg({
                            'albums_name': lambda x: list(set(x)),
                            'artists_name': lambda x: list(set(x)),
                            'genres_name': lambda x: list(set(x))
                        }).reset_index()

In [5]:
result

,id,name,albums_name,artists_name,genres_name
0,26,Complimentary Me,[Taller Children],[Elizabeth & the Catapult],"[folk, pop]"
1,38,Momma's Boy,[Taller Children],[Elizabeth & the Catapult],"[folk, pop]"
2,135,Atticus,[Wild Young Hearts],[Noisettes],[pop]
3,136,24 Hours,[Wild Young Hearts],[Noisettes],[pop]
4,138,Don't Upset The Rhythm (Go Baby Go),"[Wild Young Hearts, Don't Upset The Rhythm, Hi...",[Noisettes],[pop]
...,...,...,...,...,...
932659,101478482,На лицо,[На лицо],[FLESH],"[rusrap, rap]"
932660,101490148,Без капли мысли,[Без капли мысли],[Даня Милохин],"[pop, ruspop]"
932661,101493057,SKITTLES,[SKITTLES],[WhyBaby?],"[foreignrap, rap]"
932662,101495927,Москва,[Москва],[Yanix],"[rusrap, rap]"


In [6]:
def form_results(lst):
    return result[result['id'].isin(lst)]["name"]

In [7]:
form_results( [
    37615
])

5903    Song From A Secret Garden
Name: name, dtype: object

In [16]:
def test_root_endpoint():
    """Тест корневой точки"""
    logger.info("Тестирование корневой точки")
    response = requests.get(URL)
    if response.status_code != 200:
        logger.error(f"Ожидался статус 200, но получили {response.status_code}")
        return False
    logger.info(f"Получен статус {response.status_code}")
    return True

def test_personal_recommendations_without_history():
    """Тест для пользователя с персональными рекомендациями, но без онлайн-истории"""
    params = {"user_id": 1}
    logger.info(f"Тестирование рекомендаций для user_id={params['user_id']}, с персональными рекомендациями, без истории")
    response = requests.post(URL + 'recommendations', headers=HEADERS, params=params)
    if response.status_code != 200:
        logger.error(f"Ожидался статус 200, но получили {response.status_code}")
        return False
    logger.info(f"Получен статус {response.status_code}")
    logger.info(f"Рекомендации для user_id={params['user_id']}:\n{form_results(response.json()['recs'])}")
    return True

def test_personal_recommendations_with_history():
    """Тест для пользователя с персональными рекомендациями и онлайн-историей"""
    params = {"user_id": 0}
    logger.info(f"Тестирование рекомендаций для user_id={params['user_id']}, с персональными рекомендациями, с историей")
    response = requests.post(URL + 'recommendations', headers=HEADERS, params=params)
    if response.status_code != 200:
        logger.error(f"Ожидался статус 200, но получили {response.status_code}")
        return False
    logger.info(f"Получен статус {response.status_code}")
    logger.info(f"Рекомендации для user_id={params['user_id']}:\n{form_results(response.json()['recs'])}")
    return True

def test_non_personal_recommendations():
    """Тест для пользователя без персональных рекомендаций"""
    params = {"user_id": 8345927}
    logger.info(f"Тестирование топ 10 популярных треков для user_id={params['user_id']}, без персональных рекомендаций, без истории")
    response = requests.post(URL + 'recommendations', headers=HEADERS, params=params)
    if response.status_code != 200:
        logger.error(f"Ожидался статус 200, но получили {response.status_code}")
        return False
    logger.info(f"Получен статус {response.status_code}")
    logger.info(f"Рекомендации для user_id={params['user_id']}:\n{form_results(response.json()['recs'])}")
    return True

In [18]:
# Запуск тестов
if not test_root_endpoint():
    logger.error("test_root_endpoint failed")
if not test_personal_recommendations_without_history():
    logger.error("test_personal_recommendations_without_history failed")
if not test_personal_recommendations_with_history():
    logger.error("test_personal_recommendations_with_history failed")
if not test_non_personal_recommendations():
    logger.error("test_non_personal_recommendations failed")

2024-09-05 21:51:20,817 - test - INFO - Тестирование корневой точки
2024-09-05 21:51:20,822 - test - INFO - Получен статус 200
2024-09-05 21:51:20,823 - test - INFO - Тестирование рекомендаций для user_id=1, с персональными рекомендациями, без истории
2024-09-05 21:51:20,826 - test - INFO - Получен статус 200
2024-09-05 21:51:20,847 - test - INFO - Рекомендации для user_id=1:
1891                     Big Big World
2276                          All Star
3650                           Du Hast
5253                      Say It Right
5903         Song From A Secret Garden
5967      Tokyo Drift (Fast & Furious)
8036                       Don't Speak
9694                     The Lost Song
15574                    Пачка сигарет
305550                          Dancin
Name: name, dtype: object
2024-09-05 21:51:20,848 - test - INFO - Тестирование рекомендаций для user_id=0, с персональными рекомендациями, с историей
2024-09-05 21:51:20,852 - test - INFO - Получен статус 200
2024-09-05 21:51:20,87

In [15]:
test_non_personal_recommendations()

2024-09-05 21:50:45,820 - test - INFO - Тестирование топ 10 популярных треков для user_id=8345927, без персональных рекомендаций, без истории
2024-09-05 21:50:45,825 - test - INFO - Получен статус 200
2024-09-05 21:50:45,846 - test - INFO - Рекомендации для user_id=8345927:
5869                       Zombie
8215      Smells Like Teen Spirit
24433                        Numb
83809           Shape Of My Heart
181264          Seven Nation Army
341295             Way Down We Go
443347               Shape of You
451664                   Believer
478765                 I Got Love
607542                       Life
Name: name, dtype: object


True

In [19]:
pd.read_parquet("../parquets/events_train_sample.parquet")

,user_id,track_id,track_seq,started_at,item_id,user_id_enc
25,0,20497621,26,2022-09-15,291630,0
24,0,20232119,25,2022-09-14,287644,0
23,0,18102829,24,2022-09-13,263856,0
22,0,18042047,23,2022-09-12,262806,0
21,0,17802674,22,2022-09-10,259513,0
20,0,17802673,21,2022-09-08,259512,0
19,0,17802671,20,2022-09-06,259510,0
18,0,15769661,19,2022-09-04,233781,0
17,0,15362094,18,2022-09-02,228961,0
16,0,14370719,17,2022-08-28,219977,0
